In [12]:

import pandas as pd
import re
import boto3
from io import BytesIO
from botocore.exceptions import ClientError
import logging
import hashlib


# Variables de entornos S3
s3 = boto3.resource('s3')

# prefijo = 'naranja/dw/fac_promociones_impactados/excelIn/excel.xlsx

#prefijoExcel = 's3://datalake-prod-stage/'
#bucketOrigen = s3.Bucket('datalake-prod-stage')

#prefijoFinal= 'naranja/dw/fac_promociones_impactados/excelOut/excel.csv
#bucketDestino = 'datalake-produccion-analytics'

# buscamos el archivo
#for file in bucketOrigen.objects.filter( Prefix= prefijo):
#    df = dd.read_excel(prefijoExcel + file.key ,sheet_name='Planilla',parse_dates=['vigenciaDesde','vigenciaHasta'] )



#Cargamos el excel
df = pd.read_excel('/home/juan/anaconda3/Excel/ExcelIn/inicial.xlsx', sheet_name='Planilla',parse_dates=['VIGENCIADESDE','VIGENCIAHASTA'])



# Formateamos la fecha a DD/MM/YYYY
df['VIGENCIADESDE'] = df['VIGENCIADESDE'].dt.strftime('%d/%m/%Y')
df['VIGENCIAHASTA'] = df['VIGENCIAHASTA'].dt.strftime('%d/%m/%Y')



    
    
# Agregamos la columna estado
df['ESTADO'] = 0
df = df.fillna(value=' ')

#df = df.head(10)
#print (df.columns)


# Pasamos a minuscula el nombre y agregamos %
for row in df.itertuples():
    indice = row.Index
    
    df.iloc[(indice),7] = str(df.iloc[(indice),7].rstrip('\n'))
   # df.iloc[(indice),1] = df.iloc[(indice),1].lower()

# Concatenamos

for row in df.itertuples():
    
    indice    = row.Index
    
    # Variable provincia,localidad y CA
    provincia = str(df.iloc[(indice),5])
    localidad = str(df.iloc[(indice),6])
    CA        =  str(df.iloc[(indice),19])
    
        
    
    # pregunta si el ID del bucle es igual al ID anterior
    
    
    if row.NOMBRECOMERCIO.lower() == str(df.iloc[(indice-1) ,1]).lower() and row.PLANPRINCIPAL == str(df.iloc[(indice-1) ,9]) and str(row.PERIODOTN) == str(df.iloc[(indice-1) ,22]) and indice >= 1:     
         
        
        # Valida si dice varias provincias        
        if df.iloc[(indice-1),23] != 'Varias provincias': 
                
            
            if(re.search(provincia, (str(df.iloc[(indice-1),5])))) :
                df.iloc[(indice),23] = provincia
                
            else:
                df.iloc[(indice),23] = 'Varias provincias'
                
                
                
        else: 
            df.iloc[(indice),23] = 'Varias provincias'
            
            
     
        
        
        # Valida si dice varias localidades    
        if df.iloc[(indice-1),24] !='Varias localidades': 
           
            if(re.search((str(localidad)),(str(df.iloc[(indice-1),6])))):
                
                df.iloc[(indice),24] = localidad
                
            else:
                df.iloc[(indice),24] ='Varias localidades'
                
        else: 
            df.iloc[(indice),24] = 'Varias localidades'
        
        
        
        #Concatenamos CA en caso de ser diferentes 
        
        if(re.search( CA,(str(df.iloc[(indice-1),19])))):
            
            df.iloc[(indice),19] = (str(df.iloc[(indice-1),19]))
                
        else:
            df.iloc[(indice),19] = (str(df.iloc[(indice-1),19])) + '|' + CA
        
    
        
        
        # Concatenamos localidades   
        df.iloc[(indice),6] = (str(df.iloc[(indice-1),6]) + '|' + str(df.iloc[(indice),6]) )
        
        # Concatenamos provincias 
        df.iloc[(indice),5] = (str(df.iloc[(indice-1),5]) + '|' + provincia )
        
        # Concatenamos direcciones 
        direccion = (str(df.iloc[(indice-1),7]) + '|' + (str(df.iloc[(indice),7])) 
                            + ',' + localidad + ',' + provincia )
        direccion2 = ((str(df.iloc[(indice),7])) + ',' + localidad + ',' + provincia ).lower()
        
        print((df.iloc[(indice-1),7]).lower())
        print(direccion2)
        if (str(df.iloc[(indice-1),7]).lower()).find(direccion2) >= 0:
            df.iloc[(indice),7] = df.iloc[(indice-1),7]
        else:
            df.iloc[(indice),7] = direccion
            
        
        
        # Cambiamos el estado asignamos 2 al ultimo  y 1 al anterior
        df.iloc[indice,27]     = 2
        df.iloc[(indice-1,27)] = 1
        
        
 
    else:
        df.iloc[(indice),7]  = (str(df.iloc[(indice),7])) + ',' + (str(df.iloc[(indice),6]) ) + ',' + (str(df.iloc[(indice),5]) )
        df.iloc[(indice),23] = provincia
        df.iloc[(indice),24] = localidad
    

    
#Creacion del campo Concatenacion Hash
for row in df.itertuples():
    indice    =  row.Index
    nombre    =  df.iloc[(indice) ,1]
    plan      =  df.iloc[(indice) ,9]
    localidad =  df.iloc[(indice) ,24]
    provincia =  df.iloc[(indice) ,23]
    periodo   =  df.iloc[(indice) ,22]
    legales   =  df.iloc[(indice) ,15]
    
    logo = nombre.replace(" ","_")
    logo = nombre.lower()
    logo = logo +'.png' 
    logo = logo.replace("_.",".")
    logo = logo.replace("á","a")
    logo = logo.replace("é","e")
    logo = logo.replace("í","i")
    logo = logo.replace("ó","o")
    logo = logo.replace("ú","u")
    df.iloc[(indice),25] = logo
       


    # Buscamos los ; y reemplazamos por.

    legales = legales.replace(';','.')
    df.iloc[(indice) ,15] = legales
    
    
    df.iloc[(indice) ,26] = str(nombre) + '|' + str(plan) + '|' + str(localidad) + '|' + str(provincia) + '|' + str(periodo)
    m =  df.iloc[(indice) ,26]
    h = hashlib.md5(m.encode())
    df.iloc[(indice),0] = str(h.hexdigest())


        
#Armado de dataset de concatenados y unicos

df2 = df[df.ESTADO == 2]
df3 = df[df.ESTADO == 0]

# Quitamos columna de indice y de estado a los DF

del df2['ESTADO']
del df3['ESTADO']

    
#Limpiamos memoria
df = ''

#Concatenacion de datasets
df4 = pd.concat([df2,df3])




#print(df4.dtypes)
    
df2 = ''
df3 = ''

# Creamos campo logo

    
df4.to_excel('/home/juan/anaconda3/Excel/ExcelOut/AgrupadoHash.xlsx', sheet_name='Planilla', index=False)
df4.to_csv('/home/juan/anaconda3/Excel/ExcelOut/promo.csv', index=False , sep=';')


#buffer = StringIO()
#df4.to_csv(buffer)
#s3.Object(bucketDestino,prefijoFinal ).put(Body=buffer.getvalue())

print('Excel creado correctamente')

av. o higgins 3853,córdoba,córdoba
av. o higgins 3853,córdoba,córdoba
av. o higgins 3853,córdoba,córdoba
av. scalabrini ortiz 1285,san juan,san juan
av. o higgins 3853,córdoba,córdoba|av. scalabrini ortiz 1285,san juan,san juan
av. o higgins 3853,córdoba,córdoba
Excel creado correctamente
